In [2]:
!pip install salesforce-cdp-connector

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [14]:
# Get sklearn version and make sure that image version supports it
import sklearn
print(sklearn.__version__)
!pip install sagemaker --upgrade
# https://docs.aws.amazon.com/sagemaker/latest/dg/sklearn.html

1.0.2
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
from salesforcecdpconnector.connection import SalesforceCDPConnection

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

import sagemaker

import joblib

import boto3
from botocore.exceptions import ClientError

import json

import os
#import pickle
#import tarfile
#from sagemaker.estimator import Estimator
import time
from time import gmtime, strftime
import subprocess

role = "arn:aws:iam::768394235073:role/service-role/AmazonSageMaker-ExecutionRole-20220810T125482"
sagemaker_session = sagemaker.Session()
#Bucket for model artifacts 
bucket = sagemaker_session.default_bucket()
print(bucket)

project_name = "genie_email_rf_demo"
end_point_prefix = "genie-email-rf-demo"

secret_name = "genie/jdbc"
region_name = "us-east-1"

# Create a Secrets Manager client
session = boto3.session.Session()
region_name = session.region_name

s3 = session.resource('s3')

sagemaker_client = session.client(
    service_name='sagemaker',
    region_name=region_name
)

runtime_client = session.client(
    service_name ='sagemaker-runtime',
    region_name=region_name
)

secretsmanager_client = session.client(
    service_name='secretsmanager',
    region_name=region_name
)

try:
    get_secret_value_response = secretsmanager_client.get_secret_value(
        SecretId=secret_name
    )
except ClientError as e:
    # For a list of exceptions thrown, see
    # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    raise e

# Decrypts secret using the associated KMS key.
cred = json.loads(get_secret_value_response['SecretString'])


conn = SalesforceCDPConnection(
        cred['AppUrl'], 
        cred['user'], 
        cred['password'], 
        cred['clientId'], 
        cred['clientSecret']
    )

sagemaker-us-east-1-768394235073


In [5]:
curs = conn.cursor()
query = "SELECT engagement_type__c, engagement_count__c FROM email_engagement_insight__cio limit 900000"
curs.execute(query)
df = pd.DataFrame(curs.fetchall());
df.columns = ['engagement_type__c','engagement_count__c']
df.head(3)

,engagement_type__c,engagement_count__c
0,Send,1.000000000000000000
1,Send,2.000000000000000000
2,Send,1.000000000000000000


In [6]:
X = df.drop(["engagement_type__c"], axis=1)
y = df["engagement_type__c"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:


model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)



accuracy = accuracy_score(y_test, y_pred)
print(accuracy)


0.6918059328043673


In [8]:
# Save model artifacts to S3

with open('model.joblib', 'wb') as f:
    joblib.dump(model,f)

#Build tar file with model data + inference code 
bashCommand = "tar -cvpzf model.tar.gz model.joblib inference.py"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

#Upload tar.gz to bucket
model_artifacts = f"s3://{bucket}/{project_name}/model.tar.gz"
response = s3.meta.client.upload_file('model.tar.gz', bucket, project_name+'/model.tar.gz')


In [9]:
# retrieve sklearn image 
image_uri = sagemaker.image_uris.retrieve(
    framework="sklearn",
    region=region_name,
    version="1.0-1",
    py_version="py3",
    instance_type="ml.m5.xlarge",
)

In [10]:
#Step 1: Model Creation 
model_name = end_point_prefix + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Model name: " + model_name)
create_model_response = sagemaker_client.create_model(
    ModelName=model_name,
    Containers=[
        {
            "Image": image_uri,
            "Mode": "SingleModel",
            "ModelDataUrl": model_artifacts,
            "Environment": {'SAGEMAKER_SUBMIT_DIRECTORY': model_artifacts,
                           'SAGEMAKER_PROGRAM': 'inference.py'} 
        }
    ],
    ExecutionRoleArn=role,
)
print("Model Arn: " + create_model_response["ModelArn"])


Model name: genie-email-rf-demo2023-02-01-23-00-23
Model Arn: arn:aws:sagemaker:us-east-1:768394235073:model/genie-email-rf-demo2023-02-01-23-00-23


In [11]:
#Step 2: EPC Creation 
sklearn_epc_name = end_point_prefix + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=sklearn_epc_name,
    ProductionVariants=[
        {
            "VariantName": "sklearnvariant",
            "ModelName": model_name,
            "InstanceType": "ml.c5.large",
            "InitialInstanceCount": 1
        },
    ],
)
print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])


Endpoint Configuration Arn: arn:aws:sagemaker:us-east-1:768394235073:endpoint-config/genie-email-rf-demo2023-02-01-23-00-24


In [12]:
#Step 3: EP Creation 
endpoint_name = end_point_prefix + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
create_endpoint_response = sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=sklearn_epc_name,
)
print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])


#Monitor creation
describe_endpoint_response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)
print(describe_endpoint_response)


Endpoint Arn: arn:aws:sagemaker:us-east-1:768394235073:endpoint/genie-email-rf-demo2023-02-01-23-00-24
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService
{'EndpointName': 'genie-email-rf-demo2023-02-01-23-00-24', 'EndpointArn': 'arn:aws:sagemaker:us-east-1:768394235073:endpoint/genie-email-rf-demo2023-02-01-23-00-24', 'EndpointConfigName': 'genie-email-rf-demo2023-02-01-23-00-24', 'ProductionVariants': [{'VariantName': 'sklearnvariant', 'DeployedImages': [{'SpecifiedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:1.0-1-cpu-py3', 'ResolvedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn@sha256:08013d6ec06abb5f848882c5026a68b64dbddf1a76eaeb4d1b5c94e244869944', 'ResolutionTime': datetime.datetime(2023, 2, 1, 23, 0, 25, 240000, tzinfo=tzlocal())}], 'CurrentWeight': 1.0, 'DesiredWeight': 1.0, 'CurrentInstanceCount': 1, 'DesiredInstanceCount': 1}], 'EndpointStatus': 'InService', 'Creatio

In [13]:

#runtime_client = boto3.client('sagemaker-runtime')
content_type = "application/json"
request_body = {"Input": [['2.000000000000000000']]}
data = json.loads(json.dumps(request_body))
payload = json.dumps(data)
endpoint_name = describe_endpoint_response['EndpointName']


response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=payload)
result = json.loads(response['Body'].read().decode())['Output']
print(result)


['Send']
